<a href="https://colab.research.google.com/github/Vishakha-NITJ/flipr-Machine-Learning-5.0/blob/master/01_Problem_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [25]:
path = '/content/gdrive/My Drive/FLiPR//'

In [26]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [27]:
train = pd.read_excel(path+'Train_dataset.xlsx')
test1 = pd.read_excel(path+'Test_dataset.xlsx', sheet_name="Test_Aug10")

train_plus_test1  =  pd.concat([train, test1]).reset_index(drop=True)

In [28]:
from sklearn.utils import shuffle
train  =  shuffle(train)
test1  =  shuffle(test1)
train_plus_test1  =  shuffle(train_plus_test1)

In [29]:
# Both in Train and Test data, each row has a unique 'Stock Index'.

In [30]:
# Imputing 'General Index' with 'Index' column.
Index__General_Index__dict  =  {'BSE':38182.08, 
                                'JSE':55722,
                                'NSE':11270.15,
                                'NYSE':12765.84, 
                                'S&P 500':3351.28
                               }

train_plus_test1['General Index']  =  [Index__General_Index__dict[i]  for i in train_plus_test1['Index']]
train['General Index']             =  [Index__General_Index__dict[i]  for i in train['Index']]
test1['General Index']             =  [Index__General_Index__dict[i]  for i in test1['Index']]

# Imputing 'Covid Impact (Beta)' with 'Industry' column.
Industry__Covid_Impact__dict  =  {'Energy':0.11, 
                                  'Healthcare':0.78, 
                                  'Information Tech':0.23, 
                                  'Materials':0.03, 
                                  'Real Estate':-0.43
                                 }

train_plus_test1['Covid Impact (Beta)']  =  [Industry__Covid_Impact__dict[i]  for i in train_plus_test1['Industry']]
train['Covid Impact (Beta)']             =  [Industry__Covid_Impact__dict[i]  for i in train['Industry']]
test1['Covid Impact (Beta)']             =  [Industry__Covid_Impact__dict[i]  for i in test1['Industry']]


In [11]:
# Use PyCaret.
!pip install pycaret==2.0
from pycaret.regression import *

In [12]:
#train_sub  =  train[['NAV', 'P/E Ratio', 'Stock Price']]

In [31]:
# Train Test Split.
train_, test_  =  train.iloc[:int(0.8*len(train))], train.iloc[int(0.8*len(train)):]
train.shape, train_.shape, test_.shape

((7383, 15), (5906, 15), (1477, 15))

In [ ]:
cols  =  ['NAV', 'P/E Ratio', 'Stock Price']

# cols  =  ['NAV', 'P/E Ratio', 'Volumes Traded', 'Stock Price']    # 30, 30
# cols  =  ['NAV', 'P/E Ratio', 'P/B Ratio', 'Stock Price']    # 29, 30

# cols  =  ['NAV', 'P/E Ratio', 'Volumes Traded', 'P/B Ratio', 'Stock Price']    # 31, 31

reg  =  setup(train_[cols], target = 'Stock Price')

IntProgress(value=0, description='Processing: ', max=13)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,03:44:40
Status,. . . . . . . . . . . . . . . . . .,Preparing Data for Modeling
ETC,. . . . . . . . . . . . . . . . . .,Calculating ETC


Text(value="Following data types have been inferred automatically, if they are correct press enter to continue…

,Data Type
NAV,Numeric
P/E Ratio,Numeric
Stock Price,Label


In [ ]:
best_model = compare_models(fold=5)

In [52]:
catboost = create_model(estimator='catboost', fold=5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,17.7100,3923.1279,62.6349,0.9779,0.1376,0.0556
1,18.0201,4369.8017,66.1045,0.9749,0.1690,0.0671
2,22.0718,9111.6016,95.4547,0.9477,0.1715,0.0664
3,18.2978,4341.3286,65.8888,0.9759,0.1423,0.0569
4,17.0159,4547.4316,67.4346,0.9760,0.1403,0.0528
Mean,18.6231,5258.6583,71.5035,0.9705,0.1521,0.0598
SD,1.7765,1937.3289,12.0793,0.0114,0.0149,0.0059


In [53]:
tuned_catboost  =  tune_model(catboost, optimize='MAE', fold=5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,20.0902,4005.9149,63.2923,0.9774,0.1518,0.0708
1,21.5424,4997.0407,70.6897,0.9713,0.1863,0.0884
2,22.8776,7341.3660,85.6818,0.9578,0.1753,0.0811
3,20.2523,4204.3789,64.8412,0.9766,0.1515,0.0684
4,19.4002,4403.5020,66.3589,0.9768,0.1560,0.0700
Mean,20.8325,4990.4405,70.1728,0.9720,0.1642,0.0757
SD,1.2350,1221.3578,8.1377,0.0074,0.0141,0.0078


In [61]:
pred1  =  catboost.predict(test_[cols[:-1]])
pred2  =  tuned_catboost.predict(test_[cols[:-1]])

In [62]:
actual  =  test_['Stock Price']

In [63]:
list(zip(actual, pred1))

[(157.256162996571, 150.84046103688468),
 (1056.03068985904, 1050.0874444813405),
 (599.360246396453, 598.8491556408895),
 (42.4670939687558, 46.5403463969667),
 (125.566705084831, 128.17183077734705),
 (625.187121373785, 678.3445555022265),
 (259.582904138353, 263.4293654778861),
 (1418.82597136922, 108.9802778713709),
 (868.575451930504, 871.0863373698055),
 (1269.01414740855, 1258.2447285845135),
 (208.198137498802, 192.5577851415082),
 (729.451167761013, 723.9188776804383),
 (522.410348834779, 533.6498038115844),
 (586.746363612281, 587.8799809937581),
 (1292.77658894303, 1301.9657365878127),
 (202.971023958907, 200.66203737948433),
 (792.002520980794, 788.2820292528429),
 (376.226628418055, 386.4204797569424),
 (50.7878098113472, 52.68272037338636),
 (420.714831695735, 442.9151005863905),
 (703.302091881009, 682.0561716606462),
 (730.182550472221, 650.5455819485309),
 (1274.41904020276, 1279.3139198147312),
 (440.61532082452, 450.14664161488145),
 (445.497184538989, 435.9473647510

In [64]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(actual, pred1), mean_absolute_error(actual, pred2)

(28.498220264029626, 28.947069137663636)

In [65]:
# For    cols  =  ['NAV', 'P/E Ratio', 'Stock Price']
# 27, 29
# 23, 24
# 21, 23
# 26, 26
# 28, 28
# 28, 29
# Observation: PyCaret HyperParam Tuning is Bad everytime.
# APPROACH & SOLUTION IS RELIABLE.

In [59]:
# Maybe RF or Extra Tree Regressor  with Hyper Param Tuning
# Once try with all columns to see importance.

In [19]:
from catboost import Pool, CatBoostRegressor

train_, test_  =  train.iloc[:int(0.8*len(train))], train.iloc[int(0.8*len(train)):]
cols  =  ['NAV', 'P/E Ratio', 'Stock Price']

model = CatBoostRegressor(loss_function='MAE', verbose=False)
model.fit(Pool(train_[cols[:-1]], train_['Stock Price']))
preds = model.predict(test_[cols[:-1]])

from sklearn.metrics import mean_absolute_error
actual = test_['Stock Price']
mean_absolute_error(actual, preds)

16.937487175792008

In [32]:
# Experimenting with some more features - which seem insignificant.
train_, test_  =  train.iloc[:int(0.8*len(train))], train.iloc[int(0.8*len(train)):]
from sklearn.metrics import mean_absolute_error

for  cols  in [['NAV', 'P/E Ratio', 'Stock Price'],
              ['NAV', 'P/E Ratio', 'Put-Call Ratio','Stock Price'],
                ['NAV', 'P/E Ratio', 'Volumes Traded', 'Stock Price'],
                ['NAV', 'P/E Ratio', 'P/B Ratio', 'Stock Price'],
                ['NAV', 'P/E Ratio', 'Volumes Traded', 'P/B Ratio', 'Stock Price']]:
        print(cols)

        model = CatBoostRegressor(loss_function='MAE', verbose=False)
        model.fit(Pool(train_[cols[:-1]], train_['Stock Price']))
        preds = model.predict(test_[cols[:-1]])

        actual = test_['Stock Price']
        print(mean_absolute_error(actual, preds))

# Conclusion: Yes, they are insignificant.

['NAV', 'P/E Ratio', 'Stock Price']
16.07485380640484
['NAV', 'P/E Ratio', 'Put-Call Ratio', 'Stock Price']
16.644880552489628
['NAV', 'P/E Ratio', 'Volumes Traded', 'Stock Price']
17.18741225936513
['NAV', 'P/E Ratio', 'P/B Ratio', 'Stock Price']
16.358262080436642
['NAV', 'P/E Ratio', 'Volumes Traded', 'P/B Ratio', 'Stock Price']
16.873692891216308


In [89]:
# Experimenting with a bunch of less influential features and learning rates.
train_, test_  =  train.iloc[:int(0.8*len(train))], train.iloc[int(0.8*len(train)):]
from sklearn.metrics import mean_absolute_error

for  col  in [['NAV', 'P/E Ratio', 'Stock Price'],
                ['NAV', 'P/E Ratio', 'Volumes Traded', 'Stock Price'],
                ['NAV', 'P/E Ratio', 'P/B Ratio', 'Stock Price'],
                ['NAV', 'P/E Ratio', 'Volumes Traded', 'P/B Ratio', 'Stock Price']]:
        print(col)
        for  learning_rate  in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]:
            model = CatBoostRegressor(learning_rate=learning_rate, loss_function='MAE', verbose=False)
            model.fit(Pool(train_[cols[:-1]], train_['Stock Price']))
            preds = model.predict(test_[cols[:-1]])

            actual = test_['Stock Price']
            print(learning_rate, mean_absolute_error(actual, preds))

# Conclusion: Learning Rate that CatBoost calculates is the best, no need to experiment.

['NAV', 'P/E Ratio', 'Stock Price']
0.1 15.27536101814059
0.2 16.221812196070612
0.3 16.196127986863196
0.4 16.617971083245457
0.5 17.02936598065912
0.6 17.46014479333019
0.7 17.836785652862652
0.8 19.23032634931945
0.9 19.635302713159167
1 20.231591464507517
['NAV', 'P/E Ratio', 'Volumes Traded', 'Stock Price']
0.1 15.27536101814059
0.2 16.221812196070612
0.3 16.196127986863196
0.4 16.617971083245457
0.5 17.02936598065912
0.6 17.46014479333019
0.7 17.836785652862652
0.8 19.23032634931945
0.9 19.635302713159167
1 20.231591464507517
['NAV', 'P/E Ratio', 'P/B Ratio', 'Stock Price']
0.1 15.27536101814059
0.2 16.221812196070612
0.3 16.196127986863196
0.4 16.617971083245457
0.5 17.02936598065912
0.6 17.46014479333019
0.7 17.836785652862652
0.8 19.23032634931945
0.9 19.635302713159167
1 20.231591464507517
['NAV', 'P/E Ratio', 'Volumes Traded', 'P/B Ratio', 'Stock Price']
0.1 15.27536101814059
0.2 16.221812196070612
0.3 16.196127986863196
0.4 16.617971083245457
0.5 17.02936598065912
0.6 17.46

In [84]:
# Experimenting with all the columns.
model = CatBoostRegressor(loss_function='MAE', verbose=False)
model.fit(Pool(train_.iloc[:,3:-1], train_['Stock Price']))
preds = model.predict(test_.iloc[:,3:-1])

actual = test_['Stock Price']
print(mean_absolute_error(actual, preds))

# Conclusion: Not a good idea to give all of the feature knowing they are not contributing.

15.767894577999769


In [91]:
model = CatBoostRegressor(loss_function='MAE', verbose=False)
model.fit(Pool(train_[['NAV', 'P/E Ratio']], train_['Stock Price']))
preds = model.predict(test_[['NAV', 'P/E Ratio']])

actual = test_['Stock Price']
print(mean_absolute_error(actual, preds))

15.244748993279062


In [79]:
list(zip(actual, preds))

[(445.434265809788, 455.13839361198745),
 (551.194580720534, 554.3921532389799),
 (824.04417369074, 815.1675714855703),
 (687.449328774609, 666.5005026448148),
 (517.370267114568, 525.6728893743675),
 (144.924422734036, 140.8463740338791),
 (360.521283886924, 451.3021302969454),
 (1108.133454435, 1105.3396638699264),
 (858.499517714139, 828.1252761795931),
 (639.120244341134, 643.1641509867451),
 (1224.02470305508, 1235.330455535261),
 (674.4352110311, 672.261006523206),
 (347.203977746107, 334.51387750621524),
 (879.78029773885, 873.4492092221133),
 (120.557914418205, 125.28023012055962),
 (681.554547677072, 676.2072328269107),
 (653.96059245999, 652.446651965579),
 (247.469400498992, 247.37984504515094),
 (1032.03639605217, 1028.1212079166294),
 (1415.28190831142, 451.3021302969454),
 (168.774741094368, 169.82870523250972),
 (249.154365228779, 251.93354140718654),
 (122.29438200249, 124.14000404271371),
 (812.531852073451, 819.6208259500171),
 (255.049567053998, 217.01614650747882),


In [85]:
model.feature_importances_

array([ 0.56773813,  0.59249207, 45.5327362 , 49.19396331,  2.20652487,
        0.30908305,  0.20404843,  0.39981519,  0.1696118 ,  0.49415618,
        0.32983077])

In [86]:
train.columns[3:-1]

Index(['VWAP', 'General Index', 'NAV', 'P/E Ratio', 'Volumes Traded',
       'Inventory Turnover', 'Covid Impact (Beta)', 'Tracking Error',
       'Dollar Exchange Rate', 'Put-Call Ratio', 'P/B Ratio'],
      dtype='object')

In [92]:
# FINAL PREDICTION WITH CatBoost FOR SUBMISSION. (No need to impute, CatBoost takes care.)
train = pd.read_excel(path+'Train_dataset.xlsx')
train  =  shuffle(train)
test = pd.read_excel(path+'Test_dataset.xlsx', sheet_name="Test_Aug10")

model = CatBoostRegressor(loss_function='MAE', verbose=False)
model.fit(Pool(train[['NAV', 'P/E Ratio']], train['Stock Price']))
preds = model.predict(test[['NAV', 'P/E Ratio']])

df_submission  =  test[['Stock Index']]
df_submission['Stock Price']  =  preds

df_submission.to_csv(path+'output file 01.csv', encoding='utf-8', index=False)